<a href="https://colab.research.google.com/github/marcelounb/Deep_Learning_with_python_JasonBrownlee/blob/master/9_3_Grid_Search_Deep_Learning_Model_Parameters_Pima_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9.3 Grid Search Deep Learning Model Parameters
 The previous example showed how easy it is to wrap your deep learning model from Keras and use it in functions from the scikit-learn library. In this example we go a step further. We already know we can provide arguments to the fit() function. 
 
The function that we specify to the build_fn argument when creating the KerasClassifier wrapper can also take arguments. We can use these arguments to further customize the construction of the model. In this example we use a grid search to evaluate different conﬁgurations for our neural network model and report on the combination that provides the best estimated performance. 

The create model() function is deﬁned to take two arguments optimizer and init, both of which must have default values. This will allow us to evaluate the effect of using di↵erent optimization algorithms and weight initialization schemes for our network. After creating our model, we deﬁne arrays of values for the parameter we wish to search, speciﬁcally:

1. Optimizers for searching different weight values.
2. Initializers for preparing the network weights using di↵erent schemes.
3. Number of epochs for training the model for di↵erent number of exposures to the training dataset.
4. Batches for varying the number of samples before weight updates

The options are speciﬁed into a dictionary and passed to the conﬁguration of the GridSearchCV scikit-learn class. This class will evaluate a version of our neural network model for each combination of parameters (2⇥3⇥3⇥3) for the combinations of optimizers, initializations, epochs and batches). 

Each combination is then evaluated using the default of 3-fold stratiﬁed cross validation. That is a lot of models and a lot of computation. This is not a scheme that you want to use lightly because of the time it will take to compute. It may be useful for you to design small experiments with a smaller subset of your data that will complete in a reasonable time. 

This experiment is reasonable in this case because of the small network and the small dataset (less than 1,000 instances and 9 attributes). Finally, the performance and combination of conﬁgurations for the best model are displayed, followed by the performance of all combinations of parameters.




In [1]:
from keras.models import Sequential 
from keras.layers import Dense 
import numpy as np
# fix random seed for reproducibility 
seed = 7 
np.random.seed(seed)

Using TensorFlow backend.


In [0]:
# load pima indians dataset 
dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",") 
# split into input (X) and output (Y) variables 
X = dataset[:,0:8] 
Y = dataset[:,8]

In [3]:
X.shape

(768, 8)

In [4]:
Y.shape

(768,)

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [0]:
# Function to create model, required for KerasClassifier 
def create_model(optimizer='rmsprop' , kernel_initializer= 'glorot_uniform' ):
  # create model 
  model = Sequential() 
  model.add(Dense(12, input_dim=8, kernel_initializer=kernel_initializer , activation= 'relu' )) 
  model.add(Dense(8, kernel_initializer= 'uniform' , activation= 'relu' )) 
  model.add(Dense(1, kernel_initializer= 'uniform' , activation= 'sigmoid' ))
  # Compile model 
  model.compile(loss= 'binary_crossentropy' , optimizer=optimizer , metrics=[ 'accuracy' ])
  return model

In [0]:
# create model 
model = KerasClassifier(build_fn=create_model, verbose=0) 

In [0]:
# grid search epochs, batch size and optimizer 
optimizers = [ 'rmsprop' , 'adam' ] 
kernel_initializer = [ 'glorot_uniform' , 'normal' , 'uniform' ] 
epochs = np.array([5, 10, 15]) 
batches = np.array([5, 10, 20]) 

In [0]:
param_grid = dict(optimizer=optimizers,  kernel_initializer=kernel_initializer)  # batch_size=batches, epochs=epochs, 
grid = GridSearchCV(estimator=model, param_grid=param_grid) 
grid_result = grid.fit(X, Y) 

This might take about 5 minutes to complete on your workstation executed on the CPU. running the example shows the results below. We can see that the grid search discovered that using a uniform initialization scheme, rmsprop optimizer, 150 epochs and a batch size of 5 achieved the best cross validation score of approximately 75% on this problem.


In [12]:
# summarize results 
print(grid_result.best_score_, grid_result.best_params_) 

0.652457344532013 {'kernel_initializer': 'uniform', 'optimizer': 'adam'}


In [20]:
"""for params, mean_score, scores in grid_result.score: 
  print(scores.mean(), scores.std(), params)"""

'for params, mean_score, scores in grid_result.score: \n  print(scores.mean(), scores.std(), params)'